# Get forecast for a specific spot

In [8]:
from pysurfline import SpotForecast

In [9]:
params={
    "spotId":"5842041f4e65fad6a7708890",
    "days":7,
    "intervalHours":3,
    }
spot=SpotForecast(params,verbose=True)

-----
ForecastGetter(Type:wave, Status:200)
-----
ForecastGetter(Type:wind, Status:200)
-----
ForecastGetter(Type:tides, Status:200)
-----
ForecastGetter(Type:weather, Status:200)


In [10]:
waves=spot.get_dataframe("wave")
waves.head()

,utcOffset,surf_min,surf_max,surf_optimalScore,swells
timestamp,,,,,
2022-02-13 10:00:00,-10,1.555120,1.925840,0,"[{'height': 1.0067, 'period': 8, 'direction': ..."
2022-02-13 13:00:00,-10,1.497900,1.813900,0,"[{'height': 1.09102, 'period': 8, 'direction':..."
2022-02-13 16:00:00,-10,1.440680,1.611960,0,"[{'height': 1.18276, 'period': 8, 'direction':..."
2022-02-13 19:00:00,-10,1.150483,1.455083,0,"[{'height': 1.34372, 'period': 6, 'direction':..."
2022-02-13 22:00:00,-10,1.224333,1.368333,0,"[{'height': 0.97719, 'period': 6, 'direction':..."


In [11]:
wind=spot.get_dataframe("wind")
wind.head()

,utcOffset,speed,direction,directionType,gust,optimalScore
timestamp,,,,,,
2022-02-13 10:00:00,-10,26.17570,61.06052,Cross-shore,28.44523,0
2022-02-13 13:00:00,-10,25.13432,59.40920,Cross-shore,27.72073,0
2022-02-13 16:00:00,-10,27.02486,63.20843,Cross-shore,29.17300,0
2022-02-13 19:00:00,-10,27.23453,59.22836,Cross-shore,27.23453,0
2022-02-13 22:00:00,-10,22.07972,54.38195,Cross-shore,22.07972,0


In [12]:
tides=spot.get_dataframe("tides")
tides.head()

,utcOffset,type,height
timestamp,,,
2022-02-13 10:00:00,-10,NORMAL,0.46
2022-02-13 11:00:00,-10,NORMAL,0.50
2022-02-13 11:42:27,-10,HIGH,0.51
2022-02-13 12:00:00,-10,NORMAL,0.50
2022-02-13 13:00:00,-10,NORMAL,0.46


In [13]:
weather=spot.get_dataframe("weather")
weather.head()

,utcOffset,temperature,condition
timestamp,,,
2022-02-13 10:00:00,-10,22.56595,NIGHT_BRIEF_SHOWERS_POSSIBLE
2022-02-13 13:00:00,-10,22.27188,NIGHT_CLEAR
2022-02-13 16:00:00,-10,22.22278,NIGHT_CLEAR
2022-02-13 19:00:00,-10,22.54321,CLEAR
2022-02-13 22:00:00,-10,22.72507,CLEAR


In [14]:
spot.get_dataframe("sunlightTimes")

,midnightUTCOffset,dawn,dawnUTCOffset,sunrise,sunriseUTCOffset,sunset,sunsetUTCOffset,dusk,duskUTCOffset
midnight,,,,,,,,,
2022-02-13 10:00:00,-10,2022-02-13 16:42:46,-10,2022-02-13 17:05:35,-10,2022-02-14 04:30:04,-10,2022-02-14 04:52:52,-10
2022-02-14 10:00:00,-10,2022-02-14 16:42:13,-10,2022-02-14 17:05:00,-10,2022-02-15 04:30:36,-10,2022-02-15 04:53:23,-10
2022-02-15 10:00:00,-10,2022-02-15 16:41:38,-10,2022-02-15 17:04:23,-10,2022-02-16 04:31:08,-10,2022-02-16 04:53:53,-10
2022-02-16 10:00:00,-10,2022-02-16 16:41:03,-10,2022-02-16 17:03:46,-10,2022-02-17 04:31:39,-10,2022-02-17 04:54:22,-10
2022-02-17 10:00:00,-10,2022-02-17 16:40:26,-10,2022-02-17 17:03:07,-10,2022-02-18 04:32:10,-10,2022-02-18 04:54:51,-10
2022-02-18 10:00:00,-10,2022-02-18 16:39:48,-10,2022-02-18 17:02:28,-10,2022-02-19 04:32:40,-10,2022-02-19 04:55:20,-10
2022-02-19 10:00:00,-10,2022-02-19 16:39:10,-10,2022-02-19 17:01:48,-10,2022-02-20 04:33:10,-10,2022-02-20 04:55:48,-10
